In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
import csv


#Step 1:Login to LinkedIn

#Open Chrome and Login Linkedind login site
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

#Give the LilnkedIn username
email_field = driver.find_element(By.ID,'username')
email_field.send_keys(' username')
sleep(2)

#Give the LinkedIn password
password_field = driver.find_element(By.NAME,'session_password')
password_field.send_keys(' password ')
sleep(1)

login_field = driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button')
login_field.click()
sleep(2)


# Step2: Search for the profile we want to crawl
# Locate the search bar element

search_field = driver.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input')

# Input the search query to the search bar
search_field.send_keys('NMAMIT People')

# Search
search_field.send_keys(Keys.RETURN)

profile_ID = "/in/prg2308/"
profile_URL = "https://www.linkedin.com" + profile_ID

print(profile_URL)


#Step 3: Scrape the URL's of the profile

page_source = BeautifulSoup(driver.page_source)
profiles = page_source.find_all('a',class_= 'app-aware-link ')


def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ ='app-aware-link')
    all_profile_URL=[]
    for profile in profiles:
            profile_ID = profile.get('href')
            if "/in/" in profile_ID:
                profile_ID = profile_ID.split("/in/")[1]
                profile_ID = profile_ID.split("?")[0]
                profile_URL = "https://www.linkedin.com/in/" + profile_ID + "/"
                if profile_URL not in all_profile_URL:
                    all_profile_URL.append(profile_URL)
    return all_profile_URL
print(GetURL())


#Code to extract the data from linkedin
input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script ('window.scrollTo(0,document.body.scrollHeight)')
    sleep(2)
    next_button = driver.find_element(By.CLASS_NAME,'artdeco-pagination__button--next')
    next_button.click()
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)

res = []
[res.append(x) for x in URLs_all_page if x not in res]

print(res)
#print (URLs_all_page)
URLs_all_page=res
with open('output1.csv', 'w', newline ='') as file_output:
    headers = ['Name', 'Job_Title', 'URL']
    writer = csv.DictWriter(file_output, delimiter =',', lineterminator='\n', fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
       driver.get(linkedin_URL)
       sleep(2)
       page_source = driver.page_source 
       soup = BeautifulSoup(driver.page_source, "html.parser")
       name_div = soup.find('div',{'class':'mt2 relative'})
       name_loc = name_div.find_all('div')
       try:
          name = name_loc[0].find('h1').get_text()
       except: continue 
       print(' - Profile name is: ',name )
       
       profile_title = name_div.find('div',class_ = "text-body-medium break-words").get_text().strip()
       print(' - Profile title is: ',profile_title )
       print('\n')
      
       headers[0]=name
       headers[1]=profile_title
       headers[2]= linkedin_URL
       try:
           writer.writerow({headers[0]:name, headers[1]:profile_title, headers[2]:linkedin_URL})
       except: continue
       file_output.flush()

file_output.close()

#to append our csv file
import pandas as pd

# Load the CSV file into a pandas dataframe
df = pd.read_csv('output1.csv')

# Define a function to determine alumni status based on job title
def is_alumni(job_title):
    if 'Student at NMAM Institute of Technology' in str(job_title) or 'Student at NMAMIT' in str(job_title) or "NMAMIT'23" in str(job_title):
        return 'No'
    else:
        return 'Yes'

# Create a new column 'Alumni' using the apply() function and the is_alumni() function
df['Alumni'] = df['Job_Title'].apply(is_alumni)

# Save the updated dataframe to the original CSV file
df.to_csv('output1.csv', index=False)

#to show alumni details


alumni = df[df['Alumni']=='Yes']
alumni = alumni.drop(columns = 'Alumni')
alumni
alumni.to_csv('alumni.csv', index=False)


import sqlite3

df = pd.read_csv('alumni.csv')
# Create a SQLite database and connect to it
conn = sqlite3.connect('alumni.db')
columns = list(df.columns)
droptable = f"DROP TABLE IF EXISTS AlumniDetails"
conn.execute(droptable)

create_table_sql = f"CREATE TABLE IF NOT EXISTS AlumniDetails ({','.join([f'{col} TEXT' for col in columns])})"
conn.execute(create_table_sql)

df.to_sql('AlumniDetails', conn, if_exists='append', index=False)

alumni_det = f"SELECT * FROM AlumniDetails"
res = conn.execute(alumni_det)

for row in res:
    print(row)


https://www.linkedin.com/in/prg2308/
[]
How many pages you want to scrape: 1
['https://www.linkedin.com/in/manishshetty21/', 'https://www.linkedin.com/in/ACoAAA8f6z8BR9k5bMVqNtl-E1-_kk5y4tnb_nA/', 'https://www.linkedin.com/in/ACoAABWUF30BIzK9NWltPrMzI49JprAQLXHDhS4/', 'https://www.linkedin.com/in/ashith-k-a97b02208/', 'https://www.linkedin.com/in/ACoAAB4f33oBcvNmoUGtTumMUOLMdwlpInO1O98/', 'https://www.linkedin.com/in/srinivas-shanti/', 'https://www.linkedin.com/in/ACoAADDnOzkBgQv6k29hKEmsjXTQBTWjzBXK1pM/', 'https://www.linkedin.com/in/prg2308/', 'https://www.linkedin.com/in/ACoAAB-n5noBgZxMj4sJpcKxcEL9uPRBP2DgRNs/', 'https://www.linkedin.com/in/bhargavi-nayak/', 'https://www.linkedin.com/in/ACoAABsoTnMB5oAkVKNsbAjzdMsC-nq9QHRVmSg/', 'https://www.linkedin.com/in/sanjana-nambiar-4a2800180/', 'https://www.linkedin.com/in/ACoAAC_AdUcBIWsTrj3ROtL-VvCEUp3R0On2Fj8/', 'https://www.linkedin.com/in/ACoAADB9r6wBiI7Epp4KxEsIzQAfAry1f6jJda0/', 'https://www.linkedin.com/in/ACoAADbHUJIB3ROjyp2ZgewCv-